In [7]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

In [8]:
llm = GooglePalm(google_api_key= api_key, temperature=0.7)

In [9]:
print(llm('how to make vada paav with time calculation'))

**Ingredients**

* 1 cup besan (chickpea flour)
* 1/2 cup rice flour
* 1/2 cup semolina
* 1/2 cup finely chopped onions
* 1/2 cup finely chopped tomatoes
* 1/2 cup finely chopped green chilies
* 1/2 cup finely chopped cilantro
* 1/2 teaspoon red chili powder
* 1/2 teaspoon turmeric powder
* 1/4 teaspoon garam masala
* 1/4 teaspoon amchur powder (dry mango powder)
* 1/4 teaspoon salt
* Oil for deep frying
* 4-6 burger buns
* 2 cups of potato fries
* 2 cups of tamarind chutney
* 2 cups of mint chutney

**Instructions**

1. In a large bowl, combine the besan, rice flour, semolina, onions, tomatoes, green chilies, cilantro, red chili powder, turmeric powder, garam masala, amchur powder, and salt.
2. Add enough water to make a thick batter. The batter should be thick enough to hold its shape when dropped into hot oil.
3. Heat some oil in a deep frying pan.
4. Once the oil is hot, drop spoonfuls of the batter into the hot oil. Fry the vadas until they are golden brown on all sides.
5. Remove

## Now we are connecting with our database from MySQL

In [10]:
from langchain.utilities import SQLDatabase

In [11]:
db_user ='root'
db_password ='root'
db_host ='localhost'
db_name ='atliq_tshirts'

In [12]:
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)

In [13]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [14]:
from langchain_experimental.sql import SQLDatabaseChain

In [15]:
db_chain = SQLDatabaseChain(llm = llm, database = db, verbose = True)

C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\langchain_experimental\sql\base.py:75: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [16]:
q1 = db_chain.run('how many nike tshirts in small size')



> Entering new SQLDatabaseChain chain...
how many nike tshirts in small size
SQLQuery:SELECT count(*) FROM t_shirts WHERE brand = 'Nike' AND size = 'S'
SQLResult: [(3,)]
Answer:3
> Finished chain.


In [17]:
q1

'3'

In [18]:
q2 = db_chain.run('how much is the price of inventory for all small size tshirts?')



> Entering new SQLDatabaseChain chain...
how much is the price of inventory for all small size tshirts?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('20689'),)]
Answer:20689
> Finished chain.


in the above example we could have like wrong answer i dont how did it manage to give correct answer bt we will giver it few more  defilcult so we will decide how we can improve it 

In [19]:
# q3 = db_chain.run('if we have to sell all the levis tshorts today with discount applied how much our store will genrate')

see in this question is dificult to address like its complex query and many it is using its own genrel knowledge like usually 
wehen get this question there should be coulumns like start date of discount and end date of discount we cant give discount all days so problem was that in our database we did not have any columns like startdate and enddate theen it uses oown knowledege without looking into database so we have to solve this problem suing concept of "few shot learning" lets see how we will use it

In [20]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

q3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('17178.550000'),)]
Answer:17178.55
> Finished chain.


In [21]:
q4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('17482'),)]
Answer:17482
> Finished chain.


In [22]:
q5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE color = 'White' AND brand = 'Levi'
SQLResult: [(70,), (17,), (15,)]
Answer:17
> Finished chain.


Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [23]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('102'),)]
Answer:102
> Finished chain.


### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [24]:
few_shots = [
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': q3} ,
    
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : q4},
    
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : q5
     }
]

In [51]:
exmpl = {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : str(q5)
     }

# Embedding

In [58]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [59]:
embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [60]:
string = "How many white color Levi's shirt I have? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White',Result of the SQL query,q5"

In [61]:
emb_exmpl = embeddings.embed_query(string)

In [ ]:
emb

In [29]:
len(emb),emb[:5]

(384,
 [0.0745023638010025,
  0.09563492238521576,
  0.043689947575330734,
  0.0679817795753479,
  -0.06574059277772903])

In [30]:
to_vectorize = [''.join(i.values()) for i in few_shots]

In [31]:
to_vectorize[2]

"How many white color Levi's shirt I have?SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'Result of the SQL query17"

# VectorStore

we have to store this embeding somewhere so to store this embding we have vectorStore so we can store aur embeding into this

the job of VectorStore is to take input questions convert into embedings and find the similar looking few shots examples

In [32]:
from langchain.vectorstores import Chroma

In [33]:
# !pip install chromadb==0.4.15

In [136]:
to_vectorize

["If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n Result of the SQL query17178.55",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'Result of the SQL query17482",
 "How many white color Levi's shirt I have?SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'Result of the SQL query17"]

In [145]:
vectorstore = Chroma.from_texts(to_vectorize, 
                                embedding= embeddings, 
                                metadatas= few_shots)

ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/migration#migration-to-0416---november-7-2023 


In [143]:
texts = ["Sample text 1", "Sample text 2", "Sample text 3"]
metadatas = [{"metadata_key": "metadata_value"} for _ in range(len(texts))]
ids = ['id1', 'id2', 'id3']

# Attempt to create the VectorStore using the simplified data
vectorstore = Chroma.from_texts(texts=texts, metadatas=metadatas, ids=ids)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ACER\AppData\Local\Temp\ipykernel_2872\3705880266.py", line 6, in <module>
    vectorstore = Chroma.from_texts(texts=texts, metadatas=metadatas, ids=ids)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\langchain\vectorstores\chroma.py", line 576, in from_texts
    chroma_collection.add_texts(texts=texts, metadatas=metadatas, ids=ids)
  File "C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\langchain\vectorstores\chroma.py", line 222, in add_texts
    raise e
  File "C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\langchain\vectorstores\chroma.py", line 208, in add_texts
    self._collection.upsert(
  File "C:\ML Projects\L

becouse of some issue i am not able to use perform this code so i check there documentation and get to know that they have made some changes 

# Exploring ChromaDB 

In [97]:
import chromadb
from chromadb.config import Settings

In [98]:
import chromadb
client = chromadb.PersistentClient(path="/path/to/persist/directory")

In [99]:
collection = client.create_collection('my_info')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ACER\AppData\Local\Temp\ipykernel_2872\2779048393.py", line 1, in <module>
    collection = client.create_collection('my_info')
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\chromadb\api\client.py", line 198, in create_collection
    database=self.database,
       ^^^^^^^^^^^^^^^^^^^^
  File "C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\chromadb\telemetry\opentelemetry\__init__.py", line 127, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\chromadb\api\segment.py", line 166, in create_collection
  File "C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-package

In [103]:
# collection.add(
#     documents=to_vectorize,
#     metadatas=[
#             {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
#      'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
# (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
# group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
#  """,
#      'SQLResult': "Result of the SQL query",
#      'Answer': q3} ,
    
#      {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
#       'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
#       'SQLResult': "Result of the SQL query",
#       'Answer' : q4},
    
#     {'Question': "How many white color Levi's shirt I have?",
#      'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
#      'SQLResult': "Result of the SQL query",
#      'Answer' : q5
#      }
#     ],
#     ids=['id1', 'id2' ,'id3']
# )


In [107]:
collection.add(ids=['id1','id2','id3'],
              metadatas= few_shots,
              documents=to_vectorize)

In [105]:
# collection.delete(ids=['id1','id2','id3'],)

In [108]:
collection.peek()

{'ids': ['id1', 'id2', 'id3'],
 'embeddings': [[-0.04316724091768265,
   0.09563740342855453,
   -0.011614605784416199,
   0.009946280159056187,
   -0.07270962744951248,
   0.01896713674068451,
   0.08052719384431839,
   0.0743122398853302,
   -0.021423472091555595,
   0.02657286264002323,
   0.06828176975250244,
   -0.05572639778256416,
   0.036482278257608414,
   -0.03877401724457741,
   0.011412103660404682,
   0.05454418435692787,
   0.06070972606539726,
   -0.020781857892870903,
   -0.03905399516224861,
   -0.029982535168528557,
   -0.05494127422571182,
   -0.059788331389427185,
   0.007213791366666555,
   0.006412560120224953,
   0.023991668596863747,
   -0.005308779422193766,
   -0.030989274382591248,
   -0.01461777649819851,
   0.029098033905029297,
   -0.05476975440979004,
   -0.12221026420593262,
   0.01578749530017376,
   -0.016527611762285233,
   0.015522580593824387,
   0.0023511333856731653,
   -0.08714533597230911,
   0.025547202676534653,
   -0.08888481557369232,
   0.0

In [110]:
from langchain.prompts import SemanticSimilarityExampleSelector

In [115]:
# example_selector = SemanticSimilarityExampleSelector(vectorstore= collection, k=2)

this wont work for use coz we have used diffrent approch 

In [117]:
few_shots[1]

{'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
 'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
 'SQLResult': 'Result of the SQL query',
 'Answer': '17482'}

In [121]:
example_1 = collection.query(query_texts=["If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?"],
                             n_results=1)

In [122]:
example_1

{'ids': [['id2']],
 'distances': [[0.4239700164000545]],
 'metadatas': [[{'Answer': '17482',
    'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
    'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
    'SQLResult': 'Result of the SQL query'}]],
 'embeddings': None,
 'documents': [["If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'Result of the SQL query17482"]],
 'uris': None,
 'data': None}

## Now we have to give prompt to llm do not use any of your genral knowledge and make things up instead only use columns from the given table and from given database

In [123]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [124]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [125]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### so as you can see we langchain already provides sql prompt its really great

In [127]:
from langchain.prompts import PromptTemplate

In [128]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [126]:
from langchain.prompts import FewShotPromptTemplate

In [130]:
few = FewShotPromptTemplate(example_selector=example_1, 
                      example_prompt=example_prompt,
                      suffix=PROMPT_SUFFIX, 
                      prefix=_mysql_prompt, input_variables=['input','table_info','top_k'] 
                     )

ValidationError: 1 validation error for FewShotPromptTemplate
example_selector
  instance of BaseExampleSelector expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseExampleSelector)